<a href="https://colab.research.google.com/github/br4bit/Neural-Network-Training/blob/master/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from keras import *
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import sys
sys.path.insert(0, './drive/My Drive/DL/Face Recognition')
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from matplotlib.pyplot import *
import scipy
from inception_blocks_v2 import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

In [0]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [0]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [0]:
#Load all images in a database with new size (96,96)
#The images must be in a folder with User name for mapping in database as Name->Image vector encoding (128-d)
print("Total Params:", FRmodel.count_params())
path='./drive/My Drive/DL/Face Recognition/images/'
database={} #Use Dictionary with multiple values for each key
database = load_images(path,database,FRmodel)

In [0]:
def who_is_it2(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path,model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        for i in range(len(db_enc)): #because multiple values in dictionary with same key we have to cicle through values of each key
          # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
          dist = np.linalg.norm((encoding-db_enc[i]))
          # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
          print(name)
          print(dist)
          if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.62:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist)+"\n"+"OPEN DOOR!")
    
    return min_dist

In [0]:
# Preprocess image for fit in algorithm
captured_image = "camera2.JPG"
fname = path + captured_image
captured_image=image_resize(fname)
plt.imshow(captured_image)
who_is_it2(fname, database, FRmodel)

In [0]:
for key, db_enc in database.items():
  print(key,len(db_enc))

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='./drive/My Drive/DL/Face Recognition/images/camera.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))